In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

from context import Context
from utils.utils_chain import WrapperAddress as Address, Account, hex_to_string
from utils.utils_chain import nominated_amount
from utils.contract_retrievers import retrieve_farm_by_address
from utils.utils_chain import get_token_details_for_address, get_all_token_nonces_details_for_account
from utils.utils_tx import ESDTToken
from contracts.pair_contract import PairContract
from contracts.farm_contract import FarmContract, ClaimRewardsFarmEvent, EnterFarmEvent
from contracts.dex_proxy_contract import DexProxyContract, DexProxyClaimRewardsEvent, DexProxyEnterFarmEvent, DexProxyExitFarmEvent, DexProxyRemoveLiquidityEvent, DexProxyAddLiquidityEvent
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract

context = Context()

In [ ]:
farm_contract: FarmContract
farm_contract = context.get_contracts(config.FARMS_V2)[0]
proxy_contract: DexProxyContract
proxy_contract = context.get_contracts(config.PROXIES_V2)[0]
pair_contract: PairContract
pair_contract = context.get_contracts(config.PAIRS_V2)[0]
energy_contract: SimpleLockEnergyContract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

address setup

In [ ]:
user = Account(pem_file="~/Documents/sh1.pem")
user.address = Address("erd1a2uj4dk8l7kddvzexjzffg6hs227prcd7d7ewmhqqatjt0ur9vusn3hqa4")
user.sync_nonce(context.network_provider.proxy)

claim proxy farm rewards

In [ ]:
farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_farm_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No farm tokens")
    exit()
event = DexProxyClaimRewardsEvent(farm_contract, proxy_contract.proxy_farm_token, farm_token_nonce, farm_token_amount)
txhash = proxy_contract.claim_rewards_proxy(user, context.network_provider.proxy, event)

exit position

In [ ]:
farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_farm_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No farm tokens")
    exit()
event = DexProxyExitFarmEvent(farm_contract, proxy_contract.proxy_farm_token, farm_token_nonce, farm_token_amount // 2)
txhash = proxy_contract.exit_farm_proxy(user, context.network_provider.proxy, event)

enter position

In [ ]:
farming_token_nonce, farming_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_lp_token, user.address.bech32(), context.network_provider.proxy)
farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_farm_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No farm tokens")
    exit()
event = DexProxyEnterFarmEvent(farm_contract, 
                               proxy_contract.proxy_lp_token, farming_token_nonce, farming_token_amount,
                               proxy_contract.proxy_farm_token, farm_token_nonce, farm_token_amount)
txhash = proxy_contract.enter_farm_proxy(user, context.network_provider.proxy, event)

exit lp position

In [ ]:
farming_token_nonce, farming_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_lp_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No tokens")
    exit()
event = DexProxyRemoveLiquidityEvent(pair_contract, farming_token_amount, farming_token_nonce, 1, 1)
txhash = proxy_contract.remove_liquidity_proxy(user, context.network_provider.proxy, event)

create lp position

In [ ]:
first_token_nonce, first_token_amount, _ = get_token_details_for_address(pair_contract.firstToken, user.address.bech32(), context.network_provider.proxy)

xmex = next(locked_token for locked_token in proxy_contract.locked_tokens if "XMEX" in locked_token)
xmex_list = get_all_token_nonces_details_for_account(xmex, user.address.bech32(), context.network_provider.proxy)
biggest_xmex = {}
for xmex_found in xmex_list:
    # get the highest amount one in the list
    if int(xmex_found['balance']) > int(biggest_xmex.get('balance', 0)):
        biggest_xmex = xmex_found
second_token_nonce, second_token_amount = biggest_xmex['nonce'], int(biggest_xmex['balance'])

if first_token_amount == 0 or second_token_amount == 0:
    print(f"No tokens: first token amount: {first_token_amount}; second token amount {second_token_amount}")
    exit()
event = DexProxyAddLiquidityEvent(pair_contract, pair_contract.firstToken, first_token_nonce, first_token_amount, 1,
                                  xmex, second_token_nonce, second_token_amount, 1,)
txhash = proxy_contract.add_liquidity_proxy(user, context.network_provider.proxy, event)

exit position

In [ ]:
farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_farm_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No farm tokens")
    exit()
event = DexProxyExitFarmEvent(farm_contract, proxy_contract.proxy_farm_token, farm_token_nonce, farm_token_amount)
txhash = proxy_contract.exit_farm_proxy(user, context.network_provider.proxy, event)

Lock tokens

In [ ]:
_, mex_amount, _ = get_token_details_for_address(proxy_contract.token, user.address.bech32(), context.network_provider.proxy)

lockable_tokens = ESDTToken(proxy_contract.token, 0, mex_amount)
txhash = energy_contract.lock_tokens(user, context.network_provider.proxy, [[lockable_tokens], 720])

create lp position

In [ ]:
first_token_nonce, first_token_amount, _ = get_token_details_for_address(pair_contract.firstToken, user.address.bech32(), context.network_provider.proxy)

xmex = next(locked_token for locked_token in proxy_contract.locked_tokens if "XMEX" in locked_token)
xmex_list = get_all_token_nonces_details_for_account(xmex, user.address.bech32(), context.network_provider.proxy)
biggest_xmex = {}
for xmex_found in xmex_list:
    # get a specific xmex nonce from the list
    if int(xmex_found['nonce']) == 17:
        biggest_xmex = xmex_found
second_token_nonce, second_token_amount = biggest_xmex['nonce'], int(biggest_xmex['balance'])

if first_token_amount == 0 or second_token_amount == 0:
    print(f"No tokens: first token amount: {first_token_amount}; second token amount {second_token_amount}")
    exit()
event = DexProxyAddLiquidityEvent(pair_contract, pair_contract.firstToken, first_token_nonce, first_token_amount, 1,
                                  xmex, second_token_nonce, second_token_amount, 1,)
txhash = proxy_contract.add_liquidity_proxy(user, context.network_provider.proxy, event)

enter new position

In [ ]:
farming_token_nonce, farming_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_lp_token, user.address.bech32(), context.network_provider.proxy)
event = DexProxyEnterFarmEvent(farm_contract, 
                               proxy_contract.proxy_lp_token, farming_token_nonce, farming_token_amount // 2,
                               "", 0, 0)
txhash = proxy_contract.enter_farm_proxy(user, context.network_provider.proxy, event)

merge position

In [ ]:
farming_token_nonce, farming_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_lp_token, user.address.bech32(), context.network_provider.proxy)
farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_farm_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No farm tokens")
    exit()
event = DexProxyEnterFarmEvent(farm_contract, 
                               proxy_contract.proxy_lp_token, farming_token_nonce, farming_token_amount,
                               proxy_contract.proxy_farm_token, farm_token_nonce, farm_token_amount)
txhash = proxy_contract.enter_farm_proxy(user, context.network_provider.proxy, event)

increase proxy lp energy

In [ ]:
from utils.utils_chain import dec_to_padded_hex
epochs_increase = 360

farming_token_nonce, farming_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_lp_token, user.address.bech32(), context.network_provider.proxy)
if farming_token_nonce == 0:
    print("No tokens")
    exit()

print(f"User has {farming_token_amount} farming tokens with nonce {dec_to_padded_hex(farming_token_nonce)}")
txhash = proxy_contract.increase_proxy_lp_token_energy(user, context.network_provider.proxy, 
                                                       [[ESDTToken(proxy_contract.proxy_lp_token, farming_token_nonce, farming_token_amount)],
                                                        epochs_increase])

In [ ]:
epochs_increase = 360

txhash = proxy_contract.increase_proxy_lp_token_energy(user, context.network_provider.proxy, 
                                                       [[ESDTToken("LKLP-03a2fa", 625343, 44304399799466962)],
                                                        epochs_increase])

increase proxy farm energy

In [ ]:
from utils.utils_chain import dec_to_padded_hex
from multiversx_sdk_network_providers import ApiNetworkProvider
import time
epochs_increase = 360
token_percentage_to_use = 50
looking_for_nonce = "0"

current_user_energy = energy_contract.get_energy_for_user(context.network_provider.proxy, user.address.bech32())
print(current_user_energy)

def get_token(token: str, nonce: int = 0):
    farm_token_nonce, farm_token_amount = 0, 0

    if nonce > 0:
        # looking for a specific token nonce
        farm_token_nonce = nonce
        all_tokens = get_all_token_nonces_details_for_account(token, user.address.bech32(), context.network_provider.proxy)
        for token in all_tokens:
            if token['nonce'] == farm_token_nonce:
                farm_token_amount = int(token['balance'])
    else:
        # looking for whatever token in account
        farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(token, user.address.bech32(), context.network_provider.proxy)

    return farm_token_nonce, farm_token_amount

farm_token_nonce, farm_token_amount = get_token(proxy_contract.proxy_farm_token, int(looking_for_nonce, 16))

if farm_token_amount == 0:
    print("No tokens")
    exit()

print(f"User has {farm_token_amount} tokens with nonce {dec_to_padded_hex(farm_token_nonce)}")

mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
farm_dec_attributes, lp_dec_attributes, lktk_dec_attributes = proxy_contract.get_all_decoded_farm_token_attributes_from_api(mainnet_api, farm_token_nonce)
print(f"Underlying token unlock epoch: {lktk_dec_attributes.get('unlock_epoch')}")

used_token_amount = farm_token_amount * token_percentage_to_use // 100
txhash = proxy_contract.increase_proxy_farm_token_energy(user, context.network_provider.proxy, 
                                                       [[ESDTToken(proxy_contract.proxy_farm_token, farm_token_nonce, used_token_amount)],
                                                        epochs_increase])

time.sleep(10 if user.address.get_shard() == 1 else 50)

new_user_energy = energy_contract.get_energy_for_user(context.network_provider.proxy, user.address.bech32())
print(new_user_energy)

op_to_look_for = { # Transfer the new proxy farm token to user
        "action": "transfer",
        "sender": proxy_contract.address,
        "receiver": user.address.bech32(),
        "collection": proxy_contract.proxy_farm_token,
        "value": str(used_token_amount)
    }
new_token = get_tx_op(txhash, op_to_look_for).get('identifier')
new_farm_dec_attributes, new_lp_dec_attributes, new_lktk_dec_attributes = proxy_contract.get_all_decoded_farm_token_attributes_from_api(context.network_provider.api, int(new_token.split("-")[2], 16))
energy_difference = new_user_energy.get('amount') - current_user_energy.get('amount')

print(f"Energy difference: {energy_difference}")
print(f'New underlying token unlock epoch: {new_lktk_dec_attributes.get("unlock_epoch")}')

used_lp_tokens = int(farm_dec_attributes.get('proxy_token_amount') * used_token_amount // farm_dec_attributes.get('farm_token_amount'))
used_locked_tokens = int(lp_dec_attributes.get('locked_tokens_amount') * used_lp_tokens // lp_dec_attributes.get('lp_token_amount'))

expected_energy = (int(new_lktk_dec_attributes.get('unlock_epoch')) - int(lktk_dec_attributes.get('unlock_epoch'))) * used_locked_tokens
print(f"Expected energy: {expected_energy}")
assert energy_difference == expected_energy

Decode XMEXFARM

In [ ]:
from multiversx_sdk_network_providers import ApiNetworkProvider

token_nonce = "048c04"

mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
print(proxy_contract.get_all_decoded_farm_token_attributes_from_api(context.network_provider.api, int(token_nonce, 16)))

Decode XMEXLP

In [ ]:
from multiversx_sdk_network_providers import ApiNetworkProvider

token_nonce = "2173"

mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
print(proxy_contract.get_all_decoded_lp_token_attributes_from_api(mainnet_api, int(token_nonce, 16)))

XFARM Transaction checker

In [ ]:
from utils.utils_chain import dec_to_padded_hex

tx_hash = "1194df153273bf13b40f25eeb282a34a364109358fec5258b76024af1bd0ecf0"
mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
used_api = context.network_provider.api

# Get the transaction details
tx = context.network_provider.api.get_transaction(tx_hash)
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

# Get transaction inputs
sender = tx.sender.bech32()
payment = tx.raw_response['action']['arguments']['transfers'][0]['identifier']
payment_value = tx.raw_response['action']['arguments']['transfers'][0]['value']

xmexfarm_decode, xmexlp_decode, lktoken_decode = proxy_contract.get_all_decoded_farm_token_attributes_from_api(used_api, 
                                                                                                                   int(payment.split("-")[2], 16))

rated_lp_tokens = int(xmexfarm_decode.get('proxy_token_amount') * int(payment_value) // xmexfarm_decode.get('farm_token_amount'))
# rated_loked_tokens = int(xmexlp_decode.get('locked_tokens_amount') * (int(payment_value) / xmexfarm_decode.get('farm_token_amount')) * (xmexfarm_decode.get('proxy_token_amount') / xmexlp_decode.get('lp_token_amount')))
rated_loked_tokens = int(xmexlp_decode.get('locked_tokens_amount') * rated_lp_tokens // xmexlp_decode.get('lp_token_amount'))

expected_ops = [
    { # User transfer the farm token to the proxy contract
        "action": "transfer",
        "sender": sender,
        "receiver": proxy_contract.address,
        "identifier": payment,
        "value": str(payment_value)
    },
    { # Proxy contract burns the original proxy farm token
        "action": "burn",
        "sender": proxy_contract.address,
        "identifier": payment,
        "value": str(payment_value)
    },
    { # Proxy contract burns the original proxy lp token
        "action": "burn",
        "sender": proxy_contract.address,
        "identifier": f"{xmexfarm_decode.get('proxy_token_id')}-{dec_to_padded_hex(xmexfarm_decode.get('proxy_token_nonce'))}",
        "value": str(rated_lp_tokens)
    },
    { # Proxy contract transfers the original xmex to energy contract
        "action": "transfer",
        "sender": proxy_contract.address,
        "receiver": energy_contract.address,
        "identifier": f"{xmexlp_decode.get('locked_tokens_id')}-{dec_to_padded_hex(xmexlp_decode.get('locked_tokens_nonce'))}",
        "value": str(rated_loked_tokens)
    },
    { # Energy contract burns the original xmex
        "action": "burn",
        "sender": energy_contract.address,
        "identifier": f"{xmexlp_decode.get('locked_tokens_id')}-{dec_to_padded_hex(xmexlp_decode.get('locked_tokens_nonce'))}",
        "value": str(rated_loked_tokens)
    },
    { # Energy contract adds new xmex with increased energy
        "action": "addQuantity",
        "sender": energy_contract.address,
        "collection": xmexlp_decode.get('locked_tokens_id'),
        "value": str(rated_loked_tokens)
    },
    { # Energy contract transfers the new xmex to the proxy contract
        "action": "transfer",
        "sender": energy_contract.address,
        "receiver": proxy_contract.address,
        "collection": xmexlp_decode.get('locked_tokens_id'),
        "value": str(rated_loked_tokens)
    },
    { # Proxy contract creates new proxy lp token
        "action": "create",
        "sender": proxy_contract.address,
        "collection": proxy_contract.proxy_lp_token,
        "value": str(rated_lp_tokens)
    },
    { # Proxy contract creates the new proxy farm token
        "action": "create",
        "sender": proxy_contract.address,
        "collection": proxy_contract.proxy_farm_token,
        "value": str(payment_value)
    },
    { # Transfer the new proxy farm token to user
        "action": "transfer",
        "sender": proxy_contract.address,
        "receiver": sender,
        "collection": proxy_contract.proxy_farm_token,
        "value": str(payment_value)
    },
]

# Get and check transaction operations
ops = tx.raw_response['operations']

if len(ops) != len(expected_ops):
    print("Difference in number of operations found")

# Take each operation and match it with the ones in expected ops. Try to match only the fields expected in each expected ops. 
# Operations are unordered. If all operations match, the transaction is valid.
for op in ops:
    found = False
    matched = True
    # print(op)
    for expected_op in expected_ops:
        # print(f'Matching with {expected_op}')
        if all(op.get(key) == expected_op.get(key) for key in expected_op.keys()):
            found = True
            expected_ops.remove(expected_op)
            break
    if not found:
        print(f"Operation not matched {op}")

if len(expected_ops) == 0:
    print("All expected operations matched")
else:
    for op in expected_ops:
        print(f"Remaining operation in tx: {op}")

TX Ops finder

In [ ]:
from utils.utils_chain import dec_to_padded_hex

def get_tx_op(tx_hash: str, operation: dict) -> dict:
    used_api = context.network_provider.api

    # Get the transaction details
    tx = used_api.get_transaction(tx_hash)

    # Get and check transaction operations
    ops = tx.raw_response['operations']

    # Take each op in ops and match it with operation. Try to match only the fields expected in operation dictionary. 
    # TX Operations are unordered. If any of the operations match, return it.
    for op in ops:
        # print(f'Matching with {operation}')
        if all(op.get(key) == operation.get(key) for key in operation.keys()):
            return op

State

In [ ]:
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract

energy_contract: SimpleLockEnergyContract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

energy_contract.get_energy_for_user(context.network_provider.proxy, user.address.bech32())

XMEX Token searcher

In [ ]:
from utils.decoding_structures import XMEX_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes
from utils.utils_chain import base64_to_hex
from multiversx_sdk_network_providers import ApiNetworkProvider

searched_offset = 30

energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
locked_token = energy_contract.locked_token

# get current epoch
main_api = ApiNetworkProvider("https://api.multiversx.com")
current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
print(f'Current epoch: {current_epoch}')

for i in range(1, 10000):
    result = main_api.get_non_fungible_token(locked_token, i)
    if result:
        decoded_data = decode_merged_attributes(base64_to_hex(result.attributes), XMEX_ATTRIBUTES)
        print(i, decoded_data)
        if current_epoch < decoded_data['unlock_epoch'] < current_epoch + searched_offset:
            print(i)
        if decoded_data['unlock_epoch'] < current_epoch:
            print(i)
            break
    

XMEXFARM Token searcher

In [ ]:
from utils.decoding_structures import XMEX_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes
from utils.utils_chain import base64_to_hex
from multiversx_sdk_network_providers import ApiNetworkProvider
from utils.decoding_structures import XMEXFARM_ATTRIBUTES, XMEXLP_ATTRIBUTES, XMEX_ATTRIBUTES, LKMEX_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, dec_to_padded_hex
from multiversx_sdk_network_providers import ApiNetworkProvider
# Get token details for a given farm token

searched_offset = 0

energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
locked_token = energy_contract.locked_token
proxy_contract = context.get_contracts(config.PROXIES_V2)[0]

# get current epoch
main_api = ApiNetworkProvider('https://api.multiversx.com')
used_api = main_api
current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
print(f'Current epoch: {current_epoch}')

current_nonce = int('0472a0', 16)

for i in range(1, 10000):
    token_nonce = current_nonce - i
    print(f"Checking token {proxy_contract.proxy_farm_token}-{dec_to_padded_hex(token_nonce)} : {token_nonce}")
    
    try:
        farm_token_on_network = used_api.get_non_fungible_token(proxy_contract.proxy_farm_token, token_nonce)
    except Exception as e:
        print(f"Failed to get token {proxy_contract.proxy_farm_token} with nonce {token_nonce}")
        continue

    # Decode the farm token attributes
    decoded_xmex_farm_attributes = decode_merged_attributes(base64_to_hex(farm_token_on_network.attributes), XMEXFARM_ATTRIBUTES)
    # print(decoded_xmex_farm_attributes)

    # Decode the LP token attributes
    xmex_lp_token_id = decoded_xmex_farm_attributes.get('proxy_token_id')
    if xmex_lp_token_id != proxy_contract.proxy_lp_token:
        print(f"FAIL!!! Wrong token contained by XMEXFARM token: {xmex_lp_token_id} expected {proxy_contract.proxy_lp_token}")

    try:
        lp_token_on_network = used_api.get_non_fungible_token(xmex_lp_token_id, decoded_xmex_farm_attributes.get('proxy_token_nonce'))
    except Exception as e:
        print(f"Failed to get token {xmex_lp_token_id} with nonce {decoded_xmex_farm_attributes.get('proxy_token_nonce')}")
        continue

    decoded_xmex_lp_attributes = decode_merged_attributes(base64_to_hex(lp_token_on_network.attributes), XMEXLP_ATTRIBUTES)
    # print(decoded_xmex_lp_attributes)

    # Decode the XMEX token attributes
    xmex_token_id = decoded_xmex_lp_attributes.get('locked_tokens_id')

    xmex_token_on_network = used_api.get_non_fungible_token(xmex_token_id, decoded_xmex_lp_attributes.get('locked_tokens_nonce'))

    if "XMEX" in xmex_token_id:
        decoded_attributes = decode_merged_attributes(base64_to_hex(xmex_token_on_network.attributes), XMEX_ATTRIBUTES)
    if "LKMEX" in xmex_token_id:
        decoded_attributes = decode_merged_attributes(base64_to_hex(xmex_token_on_network.attributes), LKMEX_ATTRIBUTES)
        print(decoded_attributes)

    if decoded_attributes.get('unlock_epoch', 10000000000) < current_epoch + searched_offset:
        print(f"Found token {token_nonce} : {dec_to_padded_hex(token_nonce)} with unlock epoch: {decoded_attributes.get('unlock_epoch')}")
        continue
    

XMEXLP Token searcher

In [ ]:
from utils.decoding_structures import XMEX_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes
from utils.utils_chain import base64_to_hex
from multiversx_sdk_network_providers import ApiNetworkProvider
from utils.decoding_structures import XMEXFARM_ATTRIBUTES, XMEXLP_ATTRIBUTES, XMEX_ATTRIBUTES, LKMEX_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, dec_to_padded_hex, WrapperAddress
from multiversx_sdk_network_providers import ApiNetworkProvider
# Get token details for a given farm token

searched_offset = 360*3

energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
locked_token = energy_contract.locked_token
proxy_contract = context.get_contracts(config.PROXIES_V2)[0]

# get current epoch
main_api = ApiNetworkProvider('https://api.multiversx.com')
used_api = main_api
current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
print(f'Current epoch: {current_epoch}')

current_nonce = int('a0a3', 16)

for i in range(1, 10000):
    token_nonce = current_nonce - i
    print(f"Checking token {proxy_contract.proxy_lp_token}-{dec_to_padded_hex(token_nonce)} : {token_nonce}")

    try:
        lp_token_on_network = used_api.get_non_fungible_token(xmex_lp_token_id, token_nonce)
    except Exception as e:
        print(f"Failed to get token {xmex_lp_token_id} with nonce {token_nonce}")
        continue

    decoded_xmex_lp_attributes = decode_merged_attributes(base64_to_hex(lp_token_on_network.attributes), XMEXLP_ATTRIBUTES)
    # print(decoded_xmex_lp_attributes)

    # Decode the XMEX token attributes
    xmex_token_id = decoded_xmex_lp_attributes.get('locked_tokens_id')

    xmex_token_on_network = used_api.get_non_fungible_token(xmex_token_id, decoded_xmex_lp_attributes.get('locked_tokens_nonce'))

    if "XMEX" in xmex_token_id:
        decoded_attributes = decode_merged_attributes(base64_to_hex(xmex_token_on_network.attributes), XMEX_ATTRIBUTES)
    if "LKMEX" in xmex_token_id:
        decoded_attributes = decode_merged_attributes(base64_to_hex(xmex_token_on_network.attributes), LKMEX_ATTRIBUTES)
        print(decoded_attributes)

    if decoded_attributes.get('unlock_epoch', 10000000000) < current_epoch + searched_offset:
        try:
            users_raw = used_api.do_get_generic(f"nfts/{proxy_contract.proxy_lp_token}-{dec_to_padded_hex(token_nonce)}/accounts")
            for user in users_raw:
                if not WrapperAddress(user.get('address')).is_smart_contract():
                    print(f"Found token {token_nonce} : {dec_to_padded_hex(token_nonce)} with unlock epoch: {decoded_attributes.get('unlock_epoch')} on account {user.get('address')}")
                    break
        except Exception as e:
            print(f"Failed to get users for token {proxy_contract.proxy_lp_token}-{dec_to_padded_hex(token_nonce)}")
        
    